In [12]:
# heap
# we do not need to know how it works, we only need to know how to use it
# heap is also a list, but it can make the first element always the minimum
import heapq
heap = []
heapq.heappush(heap, 2)
print(heap)
heapq.heappush(heap, 3)
print(heap)
heapq.heappush(heap, 1)
print(heap)

print('-------------------')

# we only guarantee the first element minimum, but we cannot guarantee others
print(heapq.heappop(heap))
print(heapq.heappop(heap))
print(heapq.heappop(heap))
print(heap)

print('-------------------')

# also we can let an arbitrary list becaome a heap
heap = [5, 6, 7, 1, 2, 8, 9, 10, 3, 4]
heapq.heapify(heap) # we let the list become a heap
print(heap)

# why do we use heap? We will talk about a concept called time complexity.
# let n be the size of input, and let f(n) be a function of n.
# We can say an algorithm is O(f(n)), if there exsits a constant c, so that
# the running time T <= c * f(n)
# for exmaple, to calculate the sum of a list, is O(n) because we need to do a simple iteration
# also, the find the minimum or maximum value of a list, it is also O(n)
# why do we use heap instead of finding the minimum value each time?
# The answer is, a heap can maintain the minimum value in O(log(n)) time, here log is log_2(n)
# let's recall, log(4) = 2, log(1024) = 10, much faster than O(n)
# of course, in the exam, we will not require you to calculate time complexity, here you only need to know
# why we use heap, the answer is time complexity
# what if we want to maintain the largest value? It's easy, we can let all the values * (-1)

[2]
[2, 3]
[1, 3, 2]
-------------------
1
2
3
[]
-------------------
[1, 2, 7, 3, 4, 8, 9, 10, 6, 5]


In [18]:
# graph
# graph includes nodes and edges, and it is similar to tree
# The only difference between them is:
# in a graph, there may be cycle, and there may be multiple ways from a node s to t
# but in a tree, there is no cycle, and there is only one way from node s to t

# graph can be divided into two parts: directed and undirected.
# in a directed graph, maybe we can only have edge a -> b, but no b -> a
# in an undirected graph, if there is an edge between a and b, then both a -> b and b -> a

class UndirectedGraph:

    # input is [(1, 2), (2, 3)] which means there is edge between node 1 and 2, node 2 and 3
    def __init__(self, input: list):
        # the structure of this dictionary is from a node to a list of neighbors
        # in the example above, it is {1 : [2], 2 : [1, 3], 3 : 2}
        self.nodes = {}
        for s, t in input:
            if s in self.nodes:
                self.nodes[s].append(t)
            else:
                self.nodes[s] = [t]
            if t in self.nodes:
                self.nodes[t].append(s)
            else:
                self.nodes[t] = [s]

    # how to search a node in a graph? we can still use BFS or DFS as the previous class, but we need to change something
    # use BFS to search whether an element belong to this graph
    # lf course we have all the nodes stored in the dictionary, but how can we start from an arbitrary node and reach it?
    def BFS_search(self, start : int, element : int) -> bool:
        # because there may be cycle in a graph, it is important to label all the nodes that we visit and do not visit again
        # why we use set instead of list? The answer is, to determine whether an element belong to a list is O(n)
        # but to determine whether an element belong to a set is O(1), no matter how large the set is
        # of course this is not always, but usually we can use this principle
        visited = set()
        if start == element:
            return True
        visited.add(start)
        nodes = self.nodes
        layer = [start]
        while layer:
            for e in layer:
                temp = []
                for neighbor in nodes[e]:
                    # notice we should only add node that is not visited
                    # than label it, no visit again
                    if neighbor not in visited:
                        temp.append(neighbor)
                        visited.add(neighbor)
            if element in temp:
                return True
            layer = temp
        return False

    def DFS_search(self, start : int, element : int) -> bool:
        # we still use recursion to solve it
        def dfs_helper(start : int, element : int, visited : set):
            if start == element:
                return True
            else:
                # each time we add the start node into the set
                visited.add(start)
                for neighbor in self.nodes[start]:
                    # if a node is not visited, we regard this as the start node
                    # as long as one neighbor we search is correct, return True
                    if neighbor not in visited:
                        result = dfs_helper(neighbor, element, visited)
                        if result == True:
                            return True
            return False
        return dfs_helper(start, element, set())

G = UndirectedGraph([(1, 2), (2, 3), (1, 3), (3, 4), (4, 5)])
print(G.nodes)

print(G.BFS_search(1, 3))
print(G.BFS_search(1, 6))

print(G.DFS_search(1, 3))
print(G.DFS_search(1, 6))

{1: [2, 3], 2: [1, 3], 3: [2, 1, 4], 4: [3, 5], 5: [4]}
True
False
True
False


In [24]:
# weighted graph, there are some numbers (usually positive integers) on the edge, representing the weight
# in a (googla) map, this may be the distance between two locations.
# so here is a prolem, how can we find the shortest path from node s to t?
# if all the weigths are the same (this time it is called unweighted graph), then BFS is ok
# but for a weighted graph, we need to use some new algorithms: Dijsktra
# this algorithm is very hard, not required to understand

class WeightedGraph:
    
    # input is [(1, 2, 2), (2, 3, 4)] which means there is edge between node 1 and 2 with weight 2, node 2 and 3 with weight 4
    def __init__(self, input):
        # s -> [(neighbor1, weight1), (neighbor2, weight2)]
        self.nodes = {}
        for s, t, w in input:
            if s in self.nodes:
                self.nodes[s].append([t, w])
            else:
                self.nodes[s] = [[t, w]]
            if t in self.nodes:
                self.nodes[t].append([s, w])
            else:
                self.nodes[t] = [[s, w]]
    
    def find_shortest_path(self, start, end):
        result = {} # start : 0, to reach start point is 0 distance
        dic = self.nodes
        heap = [(0, start)] # if the element is a list or turple, then first compare the first element, then second
        while len(result) < len(dic):
            while heap[0][1] in result:
                heapq.heappop(heap)
            distance, node = heapq.heappop(heap)
            # while heap and heap[0][1] == node:
            #     heapq.heappop(heap)
            result[node] = distance
            for neighbor, weight in dic[node]:
                if neighbor not in result:
                    heapq.heappush(heap, (distance + weight, neighbor))
            # print(heap)
        return result[end]
            

G = WeightedGraph([(1, 2, 2), (2, 3, 4), (1, 3, 3), (3, 4, 1), (4, 5, 1)])
print(G.nodes)
print(G.find_shortest_path(1, 5)) # 1 -> 3 -> 4 -> 5

{1: [[2, 2], [3, 3]], 2: [[1, 2], [3, 4]], 3: [[2, 4], [1, 3], [4, 1]], 4: [[3, 1], [5, 1]], 5: [[4, 1]]}
5
